In [ ]:
# 필수 항목 임포트
import os
import cv2
import yaml
import random
import kagglehub
import pandas as pd
from PIL import Image
from shutil import copy2
!pip install ultralytics
from ultralytics import YOLO
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from IPython.display import Image, display

In [ ]:
# Download latest version
path = kagglehub.dataset_download("chitholian/annotated-potholes-dataset")

print("Path to dataset files:", path)

## 전처리

In [ ]:
# --------------------------------------------
# 사용자 설정
# --------------------------------------------
input_dir = os.path.join(path, 'annotated-images')  # 원본 이미지+XML 경로
resize_output_base = '/content/pothole_dataset_resize'
final_output_base = '/content/pothole_dataset'

IMG_SIZE = 416
TRAIN_RATIO = 0.8

# --------------------------------------------
# 폴더 생성 함수
# --------------------------------------------
def ensure_dirs(dir_list):
    for d in dir_list:
        os.makedirs(d, exist_ok=True)

# --------------------------------------------
# XML → YOLO 변환 함수
# --------------------------------------------
def convert_xml_to_yolo(xml_path, orig_w, orig_h):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    lines = []

    for obj in root.iter('object'):
        cls = obj.find('name').text
        xmlbox = obj.find('bndbox')

        xmin = int(xmlbox.find('xmin').text)
        ymin = int(xmlbox.find('ymin').text)
        xmax = int(xmlbox.find('xmax').text)
        ymax = int(xmlbox.find('ymax').text)

        x_center = ((xmin + xmax) / 2) / orig_w
        y_center = ((ymin + ymax) / 2) / orig_h
        box_w = (xmax - xmin) / orig_w
        box_h = (ymax - ymin) / orig_h

        # 클래스 고정: pothole = 0
        lines.append(f"0 {x_center} {y_center} {box_w} {box_h}\n")

    return lines

# --------------------------------------------
# 1) 이미지 리사이즈 + YOLO 라벨 생성
# --------------------------------------------
image_resize_dir = os.path.join(resize_output_base, 'images')
label_resize_dir = os.path.join(resize_output_base, 'labels')
ensure_dirs([image_resize_dir, label_resize_dir])

for file_name in os.listdir(input_dir):
    if not file_name.endswith('.jpg'):
        continue

    base = os.path.splitext(file_name)[0]
    img_path = os.path.join(input_dir, file_name)
    xml_path = os.path.join(input_dir, base + '.xml')

    if not os.path.exists(xml_path):
        continue

    img = Image.open(img_path)
    orig_w, orig_h = img.size

    img = img.resize((IMG_SIZE, IMG_SIZE))
    img.save(os.path.join(image_resize_dir, file_name))

    yolo_lines = convert_xml_to_yolo(xml_path, orig_w, orig_h)

    with open(os.path.join(label_resize_dir, base + '.txt'), 'w') as f:
        f.writelines(yolo_lines)

print("이미지 리사이즈 및 YOLO 라벨 변환 완료")

# --------------------------------------------
# 2) train/val 분할 및 복사
# --------------------------------------------
train_img_dir = os.path.join(final_output_base, 'images/train')
val_img_dir = os.path.join(final_output_base, 'images/val')
train_label_dir = os.path.join(final_output_base, 'labels/train')
val_label_dir = os.path.join(final_output_base, 'labels/val')
ensure_dirs([train_img_dir, val_img_dir, train_label_dir, val_label_dir])

all_files = [f for f in os.listdir(image_resize_dir) if f.endswith('.jpg')]
random.shuffle(all_files)

train_size = int(len(all_files) * TRAIN_RATIO)
train_files = all_files[:train_size]
val_files = all_files[train_size:]

def copy_split(file_list, split_img_dir, split_label_dir):
    for file_name in file_list:
        base = os.path.splitext(file_name)[0]

        src_img = os.path.join(image_resize_dir, file_name)
        src_label = os.path.join(label_resize_dir, base + '.txt')

        if not os.path.exists(src_label):
            print(f"경고: 라벨 누락 → {src_label}")
            continue

        copy2(src_img, os.path.join(split_img_dir, file_name))
        copy2(src_label, os.path.join(split_label_dir, base + '.txt'))

copy_split(train_files, train_img_dir, train_label_dir)
copy_split(val_files, val_img_dir, val_label_dir)

print("train/val 데이터셋 구성 완료")
print("최종 데이터 경로:", final_output_base)


## 파일 생성

In [ ]:
# --------------------------------------------
# 헉습 설정 생성
# --------------------------------------------
data_yaml = """
train: /content/pothole_dataset/images/train
val: /content/pothole_dataset/images/val

nc: 1
names: ['pothole']
"""

with open('/content/data.yaml', 'w') as f:
    f.write(data_yaml)

print("data.yaml 생성 완료")

In [ ]:
# --------------------------------------------
# 수동 최적 하이퍼파라미터 파일 생성
# --------------------------------------------
best_hyperparameters = """
lr0: 0.01199
lrf: 0.01
momentum: 0.937
weight_decay: 0.00061
warmup_epochs: 4.18245
warmup_momentum: 0.751
box: 7.83165
cls: 0.45031
dfl: 1.84738
hsv_h: 0.01709
hsv_s: 0.70526
hsv_v: 0.34714
degrees: 0.0
translate: 0.09085
scale: 0.56171
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
bgr: 0.0
mosaic: 1.0
mixup: 0.0
cutmix: 0.0
copy_paste: 0.0
close_mosaic: 9.0
"""

with open('/content/best_hyperparameters.yaml', 'w') as f:
    f.write(best_hyperparameters)

print("best_hyperparameters.yaml 생성 완료")

## 하이퍼파라미터 튜닝

In [ ]:
# 모델 불러오기
model = YOLO("yolo11s.pt")

# 2️ 자동 하이퍼파라미터 탐색 실행
results = model.tune(
    data="/content/data.yaml",  # 데이터 경로
    epochs=10,                  # 학습 반복 횟수
    imgsz=416,                  # 이미지 크기
    iterations=30,              # 시도할 조합 수 (10회 정도가 Colab에 적당)
    exist_ok=True
)

# 3️ 최적 하이퍼파라미터 결과 경로 확인
print("튜닝 완료")
print("Best hyperparameters saved at: /content/runs/tune/exp/hyp_best.yaml")

In [ ]:
# CSV 파일 경로 지정
csv_path = "/content/runs/detect/tune/tune_results.csv"

# SV 읽기
df = pd.read_csv(csv_path)

# fitness 기준 내림차순 정렬
df_sorted = df.sort_values(by="fitness", ascending=False)

# 상위 10개 조합만 보기
print("상위 10개 하이퍼파라미터 조합")
df_sorted.head(10)


## 학습

In [ ]:
# 모델 불러오기
model = YOLO("yolo11s.pt")


# 최적 하이퍼파라미터 파일 경로
best_hyp_path = "/content/best_hyperparameters.yaml"

# AML 파일을 딕셔너리 형태로 불러오기
with open(best_hyp_path, 'r') as f:
    best_hyp = yaml.safe_load(f)

# close_mosaic 값을 int로 변환
if 'close_mosaic' in best_hyp:
    best_hyp['close_mosaic'] = int(best_hyp['close_mosaic'])

print("불러온 최적 하이퍼파라미터:")
for k, v in best_hyp.items():
    print(f"{k}: {v}")

model.train(
    data="/content/data.yaml",
    epochs=30,        # 학습 반복 횟수
    imgsz=416,        # 이미지 크기
    batch=16,         # 배치 크기
    **best_hyp,  # 하이퍼파라미터 파일 지정
    project="results",  # 결과 저장 폴더
    name="best_tuned",  # 실험 이름
)